<a href="https://colab.research.google.com/github/diana-rahma/Machine-Learning_Ganjil25_11/blob/main/JS11_Klasifikasi_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### TUGAS PRAKTIKUM


1. Buatlah model SVM dengan menggunakan data voice.csv dengan ketentuan,

- Split data dengan rasio 70:30 dan 80:20 untuk setiap model yang akan dibangun.
- Gunakan model dengan kernel linier.
- Gunakan model dengan kernel polynomial.
- Gunakan model dengan kernel RBF.
- Tabulasikan performansi setiap split dan kernel berdasarkan metrik akurasi.

2. Gunakan data pada praktikum 5 untuk membuat model klasifikasi siang dan malam menggunakan SVM dengan kernel RBF menggunakan fitur histrogram. Gunakan rasio 80:20. Anda dapat bereksperimen dengan hyperparameter tunning dari kernel RBF. Catat performansi akurasinya!

In [39]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


In [40]:
# Load dataset voice.csv
df = pd.read_csv("voice.csv")

# Pisahkan fitur dan label
X = df.drop(columns=["label"])
y = df["label"]


In [41]:
def evaluate_svm(X, y, test_size):
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=42, stratify=y
    )

    # Normalisasi data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    results = {}

    # 1. Linear Kernel
    svm_linear = SVC(kernel='linear')
    svm_linear.fit(X_train, y_train)
    pred_linear = svm_linear.predict(X_test)
    results['linear'] = accuracy_score(y_test, pred_linear)

    # 2. Polynomial Kernel
    svm_poly = SVC(kernel='poly', degree=3)
    svm_poly.fit(X_train, y_train)
    pred_poly = svm_poly.predict(X_test)
    results['poly'] = accuracy_score(y_test, pred_poly)

    # 3. RBF Kernel
    svm_rbf = SVC(kernel='rbf')
    svm_rbf.fit(X_train, y_train)
    pred_rbf = svm_rbf.predict(X_test)
    results['rbf'] = accuracy_score(y_test, pred_rbf)

    return results


In [42]:
results_70 = evaluate_svm(X, y, test_size=0.30)
results_80 = evaluate_svm(X, y, test_size=0.20)


In [43]:
comparison = pd.DataFrame({
    "Kernel": ["Linear", "Polynomial", "RBF"],
    "Akurasi (70:30)": [
        results_70['linear'],
        results_70['poly'],
        results_70['rbf']
    ],
    "Akurasi (80:20)": [
        results_80['linear'],
        results_80['poly'],
        results_80['rbf']
    ]
})

print(comparison)


       Kernel  Akurasi (70:30)  Akurasi (80:20)
0      Linear         0.978970         0.974763
1  Polynomial         0.960042         0.955836
2         RBF         0.983176         0.982650


### Soal 2

In [44]:
import os
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score


In [45]:
train_dir = "/content/drive/MyDrive/ML/image/training/"
test_dir = "/content/drive/MyDrive/ML/image/test/"

In [53]:
def load_images_with_histogram(folder):
    data = []
    labels = []

    for label in ["day", "night"]:
        class_path = os.path.join(folder, label)
        for file in os.listdir(class_path):
            img_path = os.path.join(class_path, file)
            img = cv2.imread(img_path)

            if img is None:
                continue

            # Convert to HSV
            hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

            # Histogram 8x8x8
            hist = cv2.calcHist([hsv], [0,1,2], None, [8,8,8], [0,180,0,256,0,256])
            hist = cv2.normalize(hist, hist).flatten()

            data.append(hist)
            labels.append(1 if label == "day" else 0) # Corrected label assignment

    return np.array(data), np.array(labels)

In [54]:
train_dir = "/content/drive/MyDrive/ML/image/training/"
test_dir = "/content/drive/MyDrive/ML/image/test/"

X, y = load_images_with_histogram(train_dir)


In [55]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)


In [56]:
# SVM RBF
svm_rbf = SVC(kernel='rbf')
svm_rbf.fit(X_train, y_train)

y_pred = svm_rbf.predict(X_test)
baseline_acc = accuracy_score(y_test, y_pred)

print("Akurasi Model SVM RBF (baseline):", baseline_acc)


Akurasi Model SVM RBF (baseline): 0.9791666666666666


In [58]:
# Hyperparameter Tuning (GridSearch RBF)
param_grid = {
    "C": [0.1, 1, 10, 100],
    "gamma": ["scale", "auto", 0.001, 0.01, 0.1, 1]
}

grid = GridSearchCV(SVC(kernel='rbf'), param_grid, cv=5, verbose=1)
grid.fit(X_train, y_train)

print("Best Params:", grid.best_params_)
print("Best Score (Train CV):", grid.best_score_)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
Best Params: {'C': 10, 'gamma': 'scale'}
Best Score (Train CV): 1.0


In [61]:
# Evaluasi Model Terbaik
best_svm = grid.best_estimator_

y_pred_best = best_svm.predict(X_test)
best_acc = accuracy_score(y_test, y_pred_best)

print("Akurasi Model SVM RBF (Setelah Tuning):", best_acc)


Akurasi Model SVM RBF (Setelah Tuning): 0.9791666666666666
